# Scraper Toutes les données du film depuis leur propre page

In [1]:
import requests

url = 'https://if-maroc.org/rabat/film/'
response = requests.get(url)
response

<Response [200]>

In [2]:
print(response.text[:100])


<!DOCTYPE html>
<html lang="fr-FR" prefix="og: http://ogp.me/ns#" class="no-js">
    <head>
       


In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

In [4]:
grille_spectacles = soup.find("section",class_="section latest archive")

In [5]:
grille_spectacles

<section class="section latest archive">
<div class="row">
<div class="col-md-12">
<div class="section-title">
<h2>Films</h2>
</div>
</div>
<article class="col-md-4">
<div class="post">
<div class="post_thumbnail">
<a href="https://if-maroc.org/rabat/film/cinema-roubaix/" title="CINÉMA / ROUBAIX, UNE LUMIÈRE">
<img src="https://if-maroc.org/rabat/wp-content/uploads/sites/18/2019/09/b9720745537z-1_20190831171340_000g26ebp3pu-1-0-1-360x204.jpg"/> </a>
</div>
<div class="post-title">
<h3><a href="https://if-maroc.org/rabat/film/cinema-roubaix/" title="CINÉMA / ROUBAIX, UNE LUMIÈRE">CINÉMA / ROUBAIX, UNE LUMIÈRE</a></h3>
</div>
<div class="post-excerpt"><p>À Roubaix, un soir de Noël, Daoud le chef de la police locale et Louis, fraîchement diplômé, font face au meurtre d’une vieille femme. Les voisines de la victime, deux jeunes femmes, Claude et Marie, sont arrêtées. Elles sont toxicomanes, alcooliques, amantes…</p>
</div>
<div class="post-info"><span class="post-date">Cinéma / 13, 14 et 1

In [6]:
spectacles = grille_spectacles.find_all('div',class_="post")

In [7]:
spectacles

[<div class="post">
 <div class="post_thumbnail">
 <a href="https://if-maroc.org/rabat/film/cinema-roubaix/" title="CINÉMA / ROUBAIX, UNE LUMIÈRE">
 <img src="https://if-maroc.org/rabat/wp-content/uploads/sites/18/2019/09/b9720745537z-1_20190831171340_000g26ebp3pu-1-0-1-360x204.jpg"/> </a>
 </div>
 <div class="post-title">
 <h3><a href="https://if-maroc.org/rabat/film/cinema-roubaix/" title="CINÉMA / ROUBAIX, UNE LUMIÈRE">CINÉMA / ROUBAIX, UNE LUMIÈRE</a></h3>
 </div>
 <div class="post-excerpt"><p>À Roubaix, un soir de Noël, Daoud le chef de la police locale et Louis, fraîchement diplômé, font face au meurtre d’une vieille femme. Les voisines de la victime, deux jeunes femmes, Claude et Marie, sont arrêtées. Elles sont toxicomanes, alcooliques, amantes…</p>
 </div>
 <div class="post-info"><span class="post-date">Cinéma / 13, 14 et 15 octobre</span><span class="post-author">Rabat</span></div> </div>,
 <div class="post">
 <div class="post_thumbnail">
 <a href="https://if-maroc.org/rabat/

In [8]:
len(spectacles)

10

### Scraping des liens

In [9]:
links = []
for i in spectacles:
    link = i.find('a')
    print(link.get('href'))
    links.append(link.get('href'))
    

https://if-maroc.org/rabat/film/cinema-roubaix/
https://if-maroc.org/rabat/film/cinema-la-source/
https://if-maroc.org/rabat/film/cinema-parasite/
https://if-maroc.org/rabat/film/le-daim/
https://if-maroc.org/rabat/film/yves/
https://if-maroc.org/rabat/film/le-jeune-ahmed/
https://if-maroc.org/rabat/film/tel-aviv-on-fire/
https://if-maroc.org/rabat/film/grace-a-dieu/
https://if-maroc.org/rabat/film/kabullywood/
https://if-maroc.org/rabat/film/la-lutte-des-classes/


Après avoir effectué le Scraping des liens, on pourra accéder à chaque lien par page. une fois à l'intérieur il faut faire correspondre chaue film avec les différents horaires et places par film

In [10]:
from time import time
from time import sleep
from random import randint

In [11]:
start_time = time()
requests = 0

In [12]:
from requests import get

In [13]:
noms = []
dates = []
heures = []
places = []
descs = []
liens = []
images = []

for i in links:
    print(i)
    response = get(i)
    
    timer = randint(8,15)
    #pause de loop
    print(timer)
    sleep(timer)
    
    #renvoyer un warning pour les non 'status code : 200'
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
    #parser en BS
    page_html = BeautifulSoup(response.text,'html.parser') 
    
    """ nom film"""
    nom = page_html.find('div',class_='col-md-6 title').text
    print(nom)
    
    """ image poster du film"""
    img = page_html.find('div',class_='post-thumbnail text-left').find('img')['src']
    print(img)
    
    """ Descriptions """
    detail = page_html.find('div',class_='content')
    
    # au ca où il n'y est aucune decription
    if detail is None:
        desc = 'NO DESCRIPTION'
        continue
        
    description = detail.find('p')        
    desc = description.text
    
    """ *** *** chaque film en a 1 à 3 différents *** ***"""
    """ date et lieu  """
    film = page_html.find('div',class_='row evenements')
    # Resserrer la recherche
    events = film.find_all('div',class_='col-md-12')
    
    #Sauter le code si on ne trouve pas les balises nécessaires
    if events is None:
        dates.append('NO DATE')
        heures.append('NO HOUR')
        places.append('NO PLACE')
        continue    
    
    #date
    d = []
    for j in events:
        print(j.find('div',class_='event-date').text)
        d.append(j.find('div',class_='event-date').text)
    
    #heure
    h = []
    for j in events:
        print(j.find('div',class_='event-hour').text)
        h.append(j.find('div',class_='event-hour').text)
    
    #place
    p = []
    for j in events:
        print((j.find('div',class_='event-place').text).strip())
        p.append((j.find('div',class_='event-place').text).strip())
        
    
    """ Regrouper tous les elts dans leurs listes"""
    k = 0
    while(k < len(d)):# selon la taille du nombre de dates
        print("Counter : "+str(k))
        noms.append(nom)
        liens.append(i)
        dates.append(d[k])
        descs.append(desc)
        images.append(img)
        heures.append(h[k])
        places.append(p[k])
        k = k+1
        
    
   
    
    
print(" *** *** Done *** *** ")

https://if-maroc.org/rabat/film/cinema-roubaix/
11

CINÉMA / ROUBAIX, UNE LUMIÈRE

https://if-maroc.org/rabat/wp-content/uploads/sites/18/2019/09/roubaix-1-360x488.jpg
 dimanche 13 oct.
 lundi 14 oct.
 mardi 15 oct.
 18:30
 18:30
 19:30
Salle Gérard Philipe                                                                     -                                                                     Rabat
Salle Gérard Philipe                                                                     -                                                                     Rabat
Cinéma Renaissance                                                                     -                                                                     Rabat
Counter : 0
Counter : 1
Counter : 2
https://if-maroc.org/rabat/film/cinema-la-source/
11

CINÉMA / LA SOURCE

https://if-maroc.org/rabat/wp-content/uploads/sites/18/2019/09/la-source-1-360x488.jpg
 dimanche 06 oct.
 lundi 07 oct.
 mardi 08 oct.
 18:30
 18:30
 19:30
Sall

## Convertir les Dates du format lettre en Numérique

Fonction de conversion

In [14]:
import datetime
import time 
import locale
locale.setlocale(locale.LC_ALL,"french")

'French_France.1252'

In [15]:
from datetime import datetime

In [16]:
def letter_to_date(num_date):
    x =datetime.now()
    #convertir en datetime object
    date_obj = datetime.strptime(num_date,' %A %d %b')
    #formater 
    return date_obj.strftime(str(x.year)+'-%m-%d')

In [17]:
datez = []
for i in dates:
    if i =='NO DATE':
        continue
    print(letter_to_date(i))
    datez.append(letter_to_date(i))

2019-10-13
2019-10-14
2019-10-15
2019-10-06
2019-10-07
2019-10-08
2019-09-29
2019-09-30
2019-10-01
2019-09-22
2019-09-23
2019-09-24
2019-09-08
2019-09-09
2019-09-10
2019-09-15
2019-09-16
2019-09-17
2019-06-23
2019-06-24
2019-06-27
2019-06-18
2019-06-09
2019-06-10
2019-06-13
2019-06-02
2019-06-03
2019-06-06


##  Tout regrouper en une seule structure

In [18]:
prog = []
for i,j,k,l,m,n,o in zip(noms,datez,heures,images,liens,descs,places):
    #Ajouter le lieu
    t = (i,j,k,l,m,n,o)
    prog.append(t)

In [19]:
prog

[('\nCINÉMA / ROUBAIX, UNE LUMIÈRE\n',
  '2019-10-13',
  ' 18:30',
  'https://if-maroc.org/rabat/wp-content/uploads/sites/18/2019/09/roubaix-1-360x488.jpg',
  'https://if-maroc.org/rabat/film/cinema-roubaix/',
  'À Roubaix, un soir de Noël, Daoud le chef de la police locale et Louis, fraîchement diplômé, font face au meurtre d’une vieille femme. Les voisines de la victime, deux jeunes femmes, Claude et Marie, sont arrêtées. Elles sont toxicomanes, alcooliques, amantes…',
  'Salle Gérard Philipe                                                                     -                                                                     Rabat'),
 ('\nCINÉMA / ROUBAIX, UNE LUMIÈRE\n',
  '2019-10-14',
  ' 18:30',
  'https://if-maroc.org/rabat/wp-content/uploads/sites/18/2019/09/roubaix-1-360x488.jpg',
  'https://if-maroc.org/rabat/film/cinema-roubaix/',
  'À Roubaix, un soir de Noël, Daoud le chef de la police locale et Louis, fraîchement diplômé, font face au meurtre d’une vieille femme. Les v

### Convertir en objet pandas

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame(prog,columns=['noms','date','heure','image','lien','description','lieu'])

In [22]:
df

,noms,date,heure,image,lien,description,lieu
0,"\nCINÉMA / ROUBAIX, UNE LUMIÈRE\n",2019-10-13,18:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-roubaix/,"À Roubaix, un soir de Noël, Daoud le chef de l...",Salle Gérard Philipe ...
1,"\nCINÉMA / ROUBAIX, UNE LUMIÈRE\n",2019-10-14,18:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-roubaix/,"À Roubaix, un soir de Noël, Daoud le chef de l...",Salle Gérard Philipe ...
2,"\nCINÉMA / ROUBAIX, UNE LUMIÈRE\n",2019-10-15,19:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-roubaix/,"À Roubaix, un soir de Noël, Daoud le chef de l...",Cinéma Renaissance ...
3,\nCINÉMA / LA SOURCE\n,2019-10-06,18:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-la-source/,"Samir vit dans une cité populaire, en banlieu...",Salle Gérard Philipe ...
4,\nCINÉMA / LA SOURCE\n,2019-10-07,18:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-la-source/,"Samir vit dans une cité populaire, en banlieu...",Salle Gérard Philipe ...
5,\nCINÉMA / LA SOURCE\n,2019-10-08,19:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-la-source/,"Samir vit dans une cité populaire, en banlieu...",Cinéma Renaissance ...
6,\nCINÉMA / PARASITE\n,2019-09-29,18:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-parasite/,"Toute la famille de Ki-taek est au chômage, et...",Salle Gérard Philipe ...
7,\nCINÉMA / PARASITE\n,2019-09-30,18:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-parasite/,"Toute la famille de Ki-taek est au chômage, et...",Salle Gérard Philipe ...
8,\nCINÉMA / PARASITE\n,2019-10-01,19:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/cinema-parasite/,"Toute la famille de Ki-taek est au chômage, et...",Cinéma Renaissance ...
9,\nLE DAIM\n,2019-09-22,18:30,https://if-maroc.org/rabat/wp-content/uploads/...,https://if-maroc.org/rabat/film/le-daim/,,Salle Gérard Philipe ...
